In [9]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import binarize
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

import numpy as np
import scipy as sp
import scipy.io as sio

import h5py

import seaborn as sns
import matplotlib.pyplot as plt

import pickle

from deepsea import *

In [11]:
valid_X, valid_Y, train_X, train_Y, test_X, test_Y = loaddata("data/deepsea/")

MemoryError: 

# Preparing training data

In [7]:
description = [i.split("\t")[1:3] for i in open("data/deepsea/features.txt").readlines()[1:-1]]

In [8]:
description

[['Dnd41', 'CTCF'],
 ['Dnd41', 'EZH2'],
 ['GM12878', 'CTCF'],
 ['GM12878', 'EZH2'],
 ['H1-hESC', 'CHD1'],
 ['H1-hESC', 'CTCF'],
 ['H1-hESC', 'EZH2'],
 ['H1-hESC', 'JARID1A'],
 ['H1-hESC', 'RBBP5'],
 ['HeLa-S3', 'CTCF'],
 ['HeLa-S3', 'EZH2'],
 ['HeLa-S3', 'Pol2(b)'],
 ['HepG2', 'CTCF'],
 ['HepG2', 'EZH2'],
 ['HMEC', 'CTCF'],
 ['HMEC', 'EZH2'],
 ['HSMM', 'CTCF'],
 ['HSMM', 'EZH2'],
 ['HSMMtube', 'CTCF'],
 ['HSMMtube', 'EZH2'],
 ['HUVEC', 'CTCF'],
 ['HUVEC', 'EZH2'],
 ['HUVEC', 'Pol2(b)'],
 ['K562', 'CHD1'],
 ['K562', 'CTCF'],
 ['K562', 'EZH2'],
 ['K562', 'HDAC1'],
 ['K562', 'HDAC2'],
 ['K562', 'HDAC6'],
 ['K562', 'p300'],
 ['K562', 'PHF8'],
 ['K562', 'PLU1'],
 ['K562', 'Pol2(b)'],
 ['K562', 'RBBP5'],
 ['K562', 'SAP30'],
 ['NH-A', 'CTCF'],
 ['NH-A', 'EZH2'],
 ['NHDF-Ad', 'CTCF'],
 ['NHDF-Ad', 'EZH2'],
 ['NHEK', 'CTCF'],
 ['NHEK', 'EZH2'],
 ['NHEK', 'Pol2(b)'],
 ['NHLF', 'CTCF'],
 ['NHLF', 'EZH2'],
 ['Osteobl', 'CTCF'],
 ['A549', 'ATF3'],
 ['A549', 'BCL3'],
 ['A549', 'CREB1'],
 ['A549', 'C